In [ ]:
"""
OpenAI Agents SDK client that talks to the FastMCP  server.

Prereqs:
    pip install openai-agents fastmcp langchain-community langchain-text-splitters


"""


In [6]:
from dotenv import load_dotenv

load_dotenv() 

True

In [1]:

import asyncio
from pathlib import Path

from agents import Agent, Runner
from agents.model_settings import ModelSettings
from agents.mcp import MCPServerSse

In [12]:
# 1) Launch the FastMCP server as a local subprocess (stdio transport)
async with MCPServerSse(
    name="FAQ MCP Server",
    params={
        "url": "http://localhost:8000/sse",
        "headers": {"X-Workspace": "FAQ"},
    },
cache_tools_list=True, # Caches the tool list for faster performance and cleaner workflow.
) as faq_server:
    # 2) Define the agent and attach the MCP server
    faq_agent = Agent(
        name="FAQ Helpdesk Assistant",
        instructions=(
            """You are a specialist in searching and retrieving company policy information.    
            RESPONSE GUIDELINES:
            - Always cite specific policy sections when possible
            - Provide clear, actionable information
            - Suggest relevant policy categories if user is unsure
            - Be honest about policy limitations
            """
        ),
        # Attach the MCP server here
        mcp_servers=[faq_server],
        model_settings=ModelSettings(
            # Use an OpenAI model; change to gpt-4.1, gpt-4o, o3-mini, etc.
            model="gpt-4.1-mini",
            tool_choice="auto",  # let the model decide when to call tools
            temperature=0.2,
        ),
    )

    # 3) Run a sample query
    user_message = "Please search policy documents for paternity leave policy including duration and eligibility requirements"
    result = await Runner.run(faq_agent, user_message)
    print("\n=== FINAL ANSWER ===\n")
    print(result.final_output)




=== FINAL ANSWER ===

Paternity Leave Policy Overview:

- Duration: Eligible employees are entitled to up to 15 days of paid paternity leave.
- Eligibility Requirements: The policy specifies that you must submit the paternity leave application form via the HR portal at https://intranet.hr/paternity-leave.

Relevant Policy Section:
- "To apply for paternity leave, submit the form via the HR portal at https://intranet.hr/paternity-leave. You are eligible for up to 15 days of paid paternity leave." (Source: policy_docs.txt)

If you need more details on eligibility (such as length of service or employment type), please specify, or consult the HR portal link provided. If you are unsure about your eligibility, the HR department can provide further clarification.

Would you like information on related policies, such as maternity leave or general leave entitlements?


In [2]:
#result